**Juan David Abaunza**

**Taller 3: Redes Neuronales para Reconocimiento de Imágenes**

**Punto 1: Preparación del entorno y datos
En este primer paso, se configura el entorno de trabajo y se importa un conjunto de datos de imágenes para su clasificación.**

In [ ]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

# Cargamos el conjunto de datos CIFAR-10
# Este dataset contiene 60,000 imágenes a color de 32x32 píxeles en 10 categorías
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalizamos los valores de píxeles al rango [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convertimos las etiquetas a formato one-hot
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Definimos los nombres de las clases para su visualización
class_names = ['Avión', 'Automóvil', 'Pájaro', 'Gato', 'Ciervo',
               'Perro', 'Rana', 'Caballo', 'Barco', 'Camión']

# Función para visualizar imágenes del dataset
def visualizar_imagenes(images, labels, class_names):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i])
        # Convertimos la etiqueta one-hot a índice
        class_index = np.argmax(labels[i])
        plt.title(class_names[class_index])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Llamar a la función para visualizar algunas imágenes
visualizar_imagenes(train_images, train_labels, class_names)

**Punto 2: Construcción de una CNN básica
Ahora se construye una red neuronal convolucional simple para clasificar las imágenes.**

In [ ]:
def crear_modelo_cnn():
    # Creamos un modelo secuencial
    modelo = models.Sequential()

    # Bloque convolucional 1
    modelo.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Dropout(0.2))

    # Bloque convolucional 2
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Dropout(0.3))

    # Bloque convolucional 3
    modelo.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Dropout(0.4))

    # Capas totalmente conectadas
    modelo.add(layers.Flatten())
    modelo.add(layers.Dense(256, activation='relu'))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Dropout(0.5))
    modelo.add(layers.Dense(10, activation='softmax'))  # Capa de salida

    # Compilamos el modelo
    modelo.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return modelo

# Creamos el modelo
modelo_cnn = crear_modelo_cnn()

# Mostramos un resumen de la arquitectura del modelo
modelo_cnn.summary()

**Punto 3: Entrenamiento del modelo
En este paso se entrena el modelo con los datos de entrenamiento y se evalúa su rendimiento.**

In [ ]:
def entrenar_modelo(modelo, train_images, train_labels, test_images, test_labels, epochs=10):
    # Entrenamos el modelo y guardamos el historial
    historial = modelo.fit(
        train_images,  # Datos de entrenamiento
        train_labels,  # Etiquetas de entrenamiento
        epochs=epochs,  # Número de épocas
        batch_size=64,  # Tamaño del lote
        validation_data=(test_images, test_labels),  # Datos de validación
        verbose=1  # Muestra progreso
    )

    # Evaluamos el modelo con los datos de prueba
    resultados = modelo.evaluate(test_images, test_labels, verbose=0)
    print(f"\nResultados finales:")
    print(f"Pérdida en datos de prueba: {resultados[0]:.4f}")
    print(f"Precisión en datos de prueba: {resultados[1]:.4f}")

    return historial

# Función para visualizar el historial de entrenamiento
def visualizar_historial(historial):
    plt.figure(figsize=(12, 5))

    # Gráfico de precisión
    plt.subplot(1, 2, 1)
    plt.plot(historial.history['accuracy'], label='Precisión entrenamiento')
    plt.plot(historial.history['val_accuracy'], label='Precisión validación')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Precisión')
    plt.legend()
    plt.grid(True)

    # Gráfico de pérdida
    plt.subplot(1, 2, 2)
    plt.plot(historial.history['loss'], label='Pérdida entrenamiento')
    plt.plot(historial.history['val_loss'], label='Pérdida validación')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Entrenamos el modelo
print("Iniciando entrenamiento...")
historial = entrenar_modelo(modelo_cnn, train_images, train_labels, test_images, test_labels, epochs=10)

# Visualizamos el historial de entrenamiento
print("\nVisualizando progreso del entrenamiento...")
visualizar_historial(historial)

**Punto 4: Predicciones y evaluación visual
Ahora se utiliza el modelo entrenado para hacer predicciones y visualizar los resultados.**

In [ ]:
def predecir_y_visualizar(modelo, images, labels, class_names, num_imagenes=5):
    # Seleccionamos algunas imágenes aleatorias
    indices = np.random.choice(len(images), num_imagenes, replace=False)

    # Predecimos las clases
    predicciones = modelo.predict(images[indices])
    clases_predichas = np.argmax(predicciones, axis=1)
    clases_reales = np.argmax(labels[indices], axis=1)

    # Visualizamos las imágenes con predicciones
    plt.figure(figsize=(12, 4 * num_imagenes))
    for i, idx in enumerate(indices):
        plt.subplot(num_imagenes, 1, i+1)
        plt.imshow(images[idx])

        # Comprobamos si la predicción es correcta
        prediccion_correcta = (clases_predichas[i] == clases_reales[i])
        color = 'green' if prediccion_correcta else 'red'

        # Mostramos etiqueta real y predicción
        plt.title(f"Real: {class_names[clases_reales[i]]} | Predicción: {class_names[clases_predichas[i]]}",
                 color=color, fontsize=12)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Llamamos a la función para visualizar algunas predicciones
predecir_y_visualizar(modelo_cnn, test_images, test_labels, class_names, num_imagenes=5)

# Calculamos la matriz de confusión
def mostrar_matriz_confusion(modelo, images, labels, class_names):
    from sklearn.metrics import confusion_matrix
    import seaborn as sns

    # Predecimos todas las imágenes
    predicciones = modelo.predict(images)
    clases_predichas = np.argmax(predicciones, axis=1)
    clases_reales = np.argmax(labels, axis=1)

    # Calculamos la matriz de confusión
    matriz_conf = confusion_matrix(clases_reales, clases_predichas)

    # Visualizamos la matriz
    plt.figure(figsize=(10, 8))
    sns.heatmap(matriz_conf, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusión', fontsize=16)
    plt.xlabel('Predicciones', fontsize=14)
    plt.ylabel('Reales', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Mostramos la matriz de confusión
mostrar_matriz_confusion(modelo_cnn, test_images, test_labels, class_names)

**Punto 5: Mejoras y experimentación
Finalmente, se experimenta con técnicas para mejorar el rendimiento del modelo entrenado.**

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def crear_modelo_mejorado():
    modelo = models.Sequential()

    # Bloque 1
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4),
                           input_shape=(32, 32, 3)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Dropout(0.2))

    # Bloque 2
    modelo.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Dropout(0.3))

    # Bloque 3
    modelo.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.MaxPooling2D((2, 2)))
    modelo.add(layers.Dropout(0.4))

    # Capas completamente conectadas
    modelo.add(layers.Flatten())
    modelo.add(layers.Dense(512, activation='relu',
                          kernel_regularizer=regularizers.l2(1e-4)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Dropout(0.5))
    modelo.add(layers.Dense(10, activation='softmax'))

    # Optimizador personalizado
    optimizer = Adam(learning_rate=0.001, decay=1e-6)

    modelo.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    return modelo

def aplicar_data_augmentation():
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        fill_mode='nearest'
    )
    return datagen

def experimentar_hiperparametros():
    # Prueba diferentes configuraciones
    configs = [
        {'filters': [64, 128, 256], 'dropout': [0.2, 0.3, 0.4], 'lr': 0.001},
        {'filters': [32, 64, 128], 'dropout': [0.3, 0.4, 0.5], 'lr': 0.0005},
        {'filters': [128, 256, 512], 'dropout': [0.1, 0.2, 0.3], 'lr': 0.002}
    ]

    for config in configs:
        print(f"\nProbando configuración: {config}")

        # Crear modelo con configuración actual
        modelo = models.Sequential()

        # Bloque 1
        modelo.add(layers.Conv2D(config['filters'][0], (3, 3), activation='relu', padding='same',
                               input_shape=(32, 32, 3)))
        modelo.add(layers.BatchNormalization())
        modelo.add(layers.MaxPooling2D((2, 2)))
        modelo.add(layers.Dropout(config['dropout'][0]))

        # Bloque 2
        modelo.add(layers.Conv2D(config['filters'][1], (3, 3), activation='relu', padding='same'))
        modelo.add(layers.BatchNormalization())
        modelo.add(layers.MaxPooling2D((2, 2)))
        modelo.add(layers.Dropout(config['dropout'][1]))

        # Bloque 3
        modelo.add(layers.Conv2D(config['filters'][2], (3, 3), activation='relu', padding='same'))
        modelo.add(layers.BatchNormalization())
        modelo.add(layers.MaxPooling2D((2, 2)))
        modelo.add(layers.Dropout(config['dropout'][2]))

        # Capas FC
        modelo.add(layers.Flatten())
        modelo.add(layers.Dense(256, activation='relu'))
        modelo.add(layers.Dropout(0.5))
        modelo.add(layers.Dense(10, activation='softmax'))

        # Compilar
        optimizer = Adam(learning_rate=config['lr'])
        modelo.compile(optimizer=optimizer,
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

        # Data augmentation
        datagen = aplicar_data_augmentation()

        # Entrenamiento con menos épocas para prueba
        history = modelo.fit(datagen.flow(train_images, train_labels, batch_size=64),
                           epochs=10,
                           validation_data=(test_images, test_labels),
                           verbose=0)

        # Mostrar resultados
        val_acc = history.history['val_accuracy'][-1]
        print(f"Precisión en validación: {val_acc:.4f}")

# Uso recomendado:
modelo_mejorado = crear_modelo_mejorado()
modelo_mejorado.summary()

# Para data augmentation:
datagen = aplicar_data_augmentation()
datagen.fit(train_images)

# Para experimentar con hiperparámetros:
# experimentar_hiperparametros()

**Taller 4: Integración de Reconocimiento de Imágenes con NLP y Chatbot**

**Punto 1: Configuración del entorno y modelos base
Primero, se configura un entorno con las herramientas necesarias para trabajar con imágenes y lenguaje natural.**

In [ ]:
# Importamos las librerías necesarias
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
import re
import os
from PIL import Image
import io

# Descargamos recursos de NLTK necesarios para NLP
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Cargamos un modelo pre-entrenado para reconocimiento de imágenes
def cargar_modelo_imagenes():
    # Usamos MobileNetV2 pre-entrenado en ImageNet (ligero y eficiente)
    modelo_base = MobileNetV2(weights='imagenet', include_top=True)
    return modelo_base

# Implementa funciones para preprocesar imágenes
def preprocesar_imagen(ruta_imagen=None, imagen_pil=None):
    """
    Preprocesa una imagen para el modelo de reconocimiento.
    Acepta una ruta de archivo o una imagen PIL.
    """
    if ruta_imagen is not None:
        # Cargar imagen desde ruta
        img = image.load_img(ruta_imagen, target_size=(224, 224))
    elif imagen_pil is not None:
        # Convertir imagen PIL al formato requerido
        if imagen_pil.mode != 'RGB':
            imagen_pil = imagen_pil.convert('RGB')
        img = imagen_pil.resize((224, 224))
    else:
        raise ValueError("Debe proporcionar ruta_imagen o imagen_pil")

    # Convertir a array numpy y agregar dimensión del batch
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocesar según lo que espera MobileNetV2
    img_preprocesada = preprocess_input(img_array)

    return img_preprocesada

# Inicializa el lemmatizador para procesamiento de texto
lemmatizer = WordNetLemmatizer()

# Cargamos el modelo de imágenes
modelo_imagenes = cargar_modelo_imagenes()
print("Modelos base cargados correctamente.")

**Punto 2: Implementación del reconocimiento de imágenes
Ahora se implementa la funcionalidad para analizar y describir imágenes.**

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

# Cargar el modelo preentrenado (ResNet50)
modelo_imagenes = ResNet50(weights='imagenet')

def preprocesar_imagen(ruta_imagen=None, imagen_pil=None):
    """
    Preprocesa una imagen para que sea compatible con el modelo ResNet50.
    """
    try:
        if ruta_imagen:
            img = Image.open(ruta_imagen)
        elif imagen_pil:
            img = imagen_pil
        else:
            return None

        # Redimensionar a 224x224 (requerido por ResNet50)
        img = img.resize((224, 224))

        # Convertir a array numpy y normalizar
        img_array = np.array(img)
        img_array = preprocess_input(img_array)  # Normalización específica de ResNet50
        img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión del batch (1, 224, 224, 3)

        return img_array
    except Exception as e:
        print(f"Error al preprocesar la imagen: {e}")
        return None

def interpretar_contexto_imagen(predicciones):
    """
    Interpreta el contexto emocional o situacional de una imagen
    basado en los objetos detectados.
    """
    contextos = []
    objetos = [etiqueta.replace('_', ' ') for (_, etiqueta, _) in predicciones]

    # Detección de personas
    if any(obj in objetos for obj in ['person', 'man', 'woman', 'child', 'baby']):
        contextos.append("Parece haber personas en la imagen.")

    # Detección de animales
    animales = ['dog', 'cat', 'bird', 'horse', 'elephant', 'tiger', 'lion']
    if any(obj in objetos for obj in animales):
        animales_detectados = [obj for obj in objetos if obj in animales]
        contextos.append(f"Hay presencia de animales como {' y '.join(animales_detectados)}.")

    # Detección de entornos naturales
    naturaleza = ['tree', 'mountain', 'beach', 'ocean', 'forest', 'sky']
    if any(obj in objetos for obj in naturaleza):
        contextos.append("La imagen tiene elementos naturales.")

    # Detección de actividades
    if 'sports' in objetos or any(obj in objetos for obj in ['ball', 'racket', 'bicycle']):
        contextos.append("Parece mostrar alguna actividad deportiva.")
    elif 'food' in objetos or any(obj in objetos for obj in ['pizza', 'burger', 'sushi']):
        contextos.append("La imagen está relacionada con comida.")

    # Determinar el contexto predominante
    if not contextos:
        return ""
    elif len(contextos) == 1:
        return contextos[0]
    else:
        return " ".join(["Además,"] + contextos[1:])

def analizar_imagen(ruta_imagen=None, imagen_pil=None):
    """
    Analiza una imagen y devuelve una descripción de lo que contiene.
    Muestra la imagen antes de realizar el análisis.
    """
    # Cargar y mostrar la imagen
    if ruta_imagen:
        img_pil = Image.open(ruta_imagen)
    elif imagen_pil:
        img_pil = imagen_pil
    else:
        return "No se proporcionó ninguna imagen."

    # Mostrar la imagen
    plt.figure(figsize=(8, 6))
    plt.imshow(img_pil)
    plt.axis('off')  # Ocultar ejes
    plt.title("Imagen a analizar")
    plt.show()

    # Preprocesar la imagen
    img_array = preprocesar_imagen(ruta_imagen, img_pil)

    if img_array is None:
        return "No se pudo procesar la imagen."

    # Realizar la predicción
    predicciones = modelo_imagenes.predict(img_array)
    etiquetas = decode_predictions(predicciones, top=3)[0]  # Top 3 predicciones

    # Formatear los resultados
    descripcion = "En esta imagen puedo ver: "
    objetos_detectados = []

    for i, (id, etiqueta, probabilidad) in enumerate(etiquetas):
        etiqueta_formateada = etiqueta.replace('_', ' ')
        if i == len(etiquetas) - 1 and len(etiquetas) > 1:
            objetos_detectados.append(f"y {etiqueta_formateada}")
        else:
            objetos_detectados.append(etiqueta_formateada)

    descripcion += ', '.join(objetos_detectados) + "."

    # Añadir interpretación contextual
    contexto = interpretar_contexto_imagen(etiquetas)
    if contexto:
        descripcion += " " + contexto

    return descripcion

# --- Ejemplo de uso ---
if __name__ == "__main__":
    # Cambia esta ruta por la de tu imagen (se debe subir a archivos, la carpeta que se ve a la izquierda en .jpg y que sea clara)
    ruta_imagen_prueba = "Pajaro.jpg"  # Ejemplo: "Perro.jpg", "Gato.jpg", etc.

    try:
        resultado = analizar_imagen(ruta_imagen_prueba)
        print(resultado)
    except Exception as e:
        print(f"Error al analizar la imagen: {e}")

**Punto 3: Desarrollo del motor NLP para el chatbot
En este paso, se implementa procesamiento de lenguaje natural para el chatbot.**

In [ ]:
import re
import spacy
from collections import defaultdict

# Cargar modelo de lenguaje para lematización (español)
nlp = spacy.load("es_core_news_sm")

# Definimos patrones de conversación básicos relacionados con imágenes y consultas
patrones_conversacion = {
    "saludos": [
        r"hola",
        r"buenos\s*d[ií]as",
        r"buenas\s*tardes",
        r"buenas\s*noches",
        r"hey",
        r"saludos"
    ],
    "preguntas_imagen": [
        r"qué\s*ves\s*en\s*(la|esta)\s*imagen",
        r"describe\s*(la|esta)\s*imagen",
        r"qué\s*hay\s*en\s*(la|esta)\s*imagen",
        r"qué\s*puedes\s*ver",
        r"analiza\s*(la|esta)\s*imagen"
    ],
    "consulta_psicologica": [
        r"me\s*siento\s*(triste|feliz|ansioso|preocupado|estresado)",
        r"tengo\s*problemas\s*con",
        r"no\s*puedo\s*(dormir|concentrarme|estudiar)",
        r"necesito\s*ayuda\s*con\s*mis\s*emociones",
        r"cómo\s*puedo\s*manejar\s*(el\s*estrés|la\s*ansiedad|la\s*depresión)"
    ],
    "consulta_academica": [
        r"no\s*entiendo\s*(este\s*tema|esta\s*materia)",
        r"cómo\s*puedo\s*estudiar\s*mejor",
        r"tengo\s*dificultades\s*con",
        r"necesito\s*ayuda\s*con\s*mis\s*estudios",
        r"cómo\s*mejorar\s*(mi\s*concentración|mi\s*memoria|mis\s*notas)"
    ],
    "despedida": [
        r"adiós",
        r"hasta\s*luego",
        r"nos\s*vemos",
        r"chao",
        r"bye"
    ]
}

# Definimos respuestas para cada categoría
respuestas = {
    "saludos": [
        "¡Hola! ¿En qué puedo ayudarte hoy?",
        "¡Saludos! Soy un asistente virtual. Puedo analizar imágenes y conversar contigo.",
        "¡Buen día! Estoy aquí para ayudarte con tus consultas e imágenes."
    ],
    "preguntas_imagen_sin_contexto": [
        "Para analizar una imagen, necesito que me la proporciones primero.",
        "No tengo ninguna imagen para analizar. ¿Podrías compartir una?",
        "Necesito ver una imagen antes de poder describirla."
    ],
    "consulta_psicologica": [
        "Entiendo cómo te sientes. ¿Podrías contarme más sobre eso?",
        "Es normal tener esos sentimientos. ¿Qué crees que los está causando?",
        "Gracias por compartir eso conmigo. Te escucho. ¿Desde cuándo te sientes así?"
    ],
    "consulta_academica": [
        "Aprender puede ser desafiante. ¿Con qué tema específico estás teniendo dificultades?",
        "Cada persona tiene su propio estilo de aprendizaje. ¿Has identificado qué métodos funcionan mejor para ti?",
        "El éxito académico requiere estrategia. ¿Has probado crear un horario de estudio?"
    ],
    "despedida": [
        "¡Hasta pronto! Fue un placer ayudarte.",
        "¡Adiós! Si necesitas más ayuda, aquí estaré.",
        "¡Que tengas un excelente día! Regresa cuando necesites apoyo."
    ],
    "default": [
        "Interesante. Cuéntame más.",
        "No estoy seguro de entender. ¿Podrías explicarlo de otra manera?",
        "Estoy aquí para ayudarte. ¿Puedes ser más específico?"
    ]
}

def procesar_texto(texto):
    """
    Procesa el texto de entrada para normalización:
    - Convierte a minúsculas
    - Elimina caracteres especiales
    - Tokeniza y lematiza
    - Elimina stopwords
    """
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar caracteres especiales (excepto letras, números y espacios)
    texto = re.sub(r'[^\w\s]', '', texto)

    # Tokenizar y lematizar
    doc = nlp(texto)
    tokens_procesados = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.text.strip()
    ]

    return tokens_procesados

def identificar_intencion(texto):
    """
    Identifica la intención del usuario basado en patrones de expresión regular
    Devuelve la categoría de intención detectada
    """
    texto = texto.lower().strip()

    # Verificar cada categoría y sus patrones
    for categoria, patrones in patrones_conversacion.items():
        for patron in patrones:
            if re.search(patron, texto):
                return categoria

    return "default"

def obtener_respuesta(intencion):
    """
    Devuelve una respuesta aleatoria para la intención detectada
    """
    from random import choice
    return choice(respuestas.get(intencion, respuestas["default"]))

def chatbot_interactivo():
    """
    Función principal para interactuar con el chatbot
    """
    print("¡Hola! Soy tu asistente virtual. Puedo:")
    print("- Analizar imágenes (describe lo que veo)")
    print("- Escucharte si necesitas apoyo psicológico")
    print("- Ayudarte con consultas académicas")
    print("Escribe 'salir' para terminar la conversación.\n")

    while True:
        entrada = input("Tú: ").strip()

        if entrada.lower() in ('salir', 'exit', 'quit'):
            print(obtener_respuesta("despedida"))
            break

        # Procesar el texto
        tokens = procesar_texto(entrada)
        print(f"(DEBUG) Tokens procesados: {tokens}")

        # Identificar intención
        intencion = identificar_intencion(entrada)
        print(f"(DEBUG) Intención detectada: {intencion}")

        # Obtener y mostrar respuesta
        respuesta = obtener_respuesta(intencion)
        print(f"Asistente: {respuesta}\n")

if __name__ == "__main__":
    # Ejemplo de uso del procesamiento de texto
    test_text = "Hola, no puedo concentrarme en mis estudios. ¿Qué me recomiendas?"
    print("\nEjemplo de procesamiento:")
    print(f"Texto original: {test_text}")
    print(f"Tokens procesados: {procesar_texto(test_text)}")
    print(f"Intención detectada: {identificar_intencion(test_text)}")

    # Iniciar chatbot interactivo
    print("\nModo interactivo:")
    chatbot_interactivo()

**Punto 4: Integración de los componentes y creación de la interfaz
Ahora se integra el reconocimiento de imágenes con el procesamiento de lenguaje natural.**

In [ ]:
from PIL import Image
import os

# Funciones auxiliares ficticias (deberías implementarlas adecuadamente)
def modelo_imagenes():
    """Simulación de un modelo de análisis de imágenes"""
    return None

def analizar_imagen(ruta_imagen=None, imagen_pil=None):
    """Función simulada para analizar imágenes"""
    if ruta_imagen:
        try:
            img = Image.open(ruta_imagen)
            return {
                "estado": "éxito",
                "descripcion": f"Imagen analizada: {os.path.basename(ruta_imagen)}",
                "detalles": {
                    "tamaño": img.size,
                    "formato": img.format,
                    "modo": img.mode
                }
            }
        except Exception as e:
            return {"estado": "error", "mensaje": str(e)}
    elif imagen_pil:
        return {
            "estado": "éxito",
            "descripcion": "Imagen PIL analizada",
            "detalles": {
                "tamaño": imagen_pil.size,
                "modo": imagen_pil.mode
            }
        }
    return {"estado": "error", "mensaje": "No se proporcionó imagen"}

def identificar_intencion(texto):
    """Identifica la intención del usuario de forma básica"""
    texto = texto.lower()
    if any(palabra in texto for palabra in ["hola", "hi", "buenos días"]):
        return "saludo"
    elif any(palabra in texto for palabra in ["adiós", "chao", "hasta luego"]):
        return "despedida"
    elif any(palabra in texto for palabra in ["qué", "cómo", "dime", "explica"]):
        return "pregunta"
    elif "imagen" in texto or "foto" in texto:
        return "consulta_imagen"
    return "otro"

class AsistenteVirtual:
    def __init__(self):
        self.modelo_imagenes = modelo_imagenes()
        self.imagen_actual = None
        self.analisis_imagen_actual = None
        self.contexto_conversacion = []

    def procesar_imagen(self, ruta_imagen=None, imagen_pil=None):
        """
        Procesa una nueva imagen y actualiza el contexto
        """
        self.analisis_imagen_actual = analizar_imagen(ruta_imagen, imagen_pil)

        if ruta_imagen:
            try:
                self.imagen_actual = Image.open(ruta_imagen)
            except:
                self.imagen_actual = None
        else:
            self.imagen_actual = imagen_pil

        return self.analisis_imagen_actual

    def responder(self, texto_usuario):
        """
        Genera una respuesta basada en el input del usuario y el contexto actual
        """
        categoria = identificar_intencion(texto_usuario)

        if categoria == "saludo":
            respuesta = "¡Hola! ¿En qué puedo ayudarte hoy?"
        elif categoria == "despedida":
            respuesta = "¡Hasta luego! Fue un placer ayudarte."
        elif categoria == "pregunta":
            if self.analisis_imagen_actual:
                respuesta = f"Actualmente estoy analizando una imagen. {self.analisis_imagen_actual['descripcion']}. ¿Quieres saber algo específico sobre ella?"
            else:
                respuesta = "Puedo ayudarte a analizar imágenes. ¿Quieres que analice alguna?"
        elif categoria == "consulta_imagen":
            if self.analisis_imagen_actual:
                detalles = self.analisis_imagen_actual.get('detalles', {})
                respuesta = (f"Actualmente tengo una imagen cargada. Detalles:\n"
                           f"Tamaño: {detalles.get('tamaño', 'N/A')}\n"
                           f"Formato: {detalles.get('formato', 'N/A')}\n"
                           f"Modo: {detalles.get('modo', 'N/A')}")
            else:
                respuesta = "No hay ninguna imagen cargada actualmente. Puedes proporcionarme una con el comando 'ver imagen: [ruta]'"
        else:
            respuesta = "No estoy seguro de cómo responder a eso. ¿Puedes reformular tu pregunta?"

        # Actualiza el contexto de la conversación
        self.contexto_conversacion.append({"usuario": texto_usuario, "respuesta": respuesta})

        return respuesta

def interfaz_cli():
    """
    Interfaz simple de línea de comandos para interactuar con el asistente
    """
    asistente = AsistenteVirtual()
    print("=== Asistente Virtual ===")
    print("Puedo analizar imágenes y conversar contigo.")
    print("Para analizar una imagen, escribe 'ver imagen: [ruta_a_la_imagen]'")
    print("Para salir, escribe 'salir'")

    while True:
        entrada = input("\nTú: ").strip()

        if entrada.lower() == "salir":
            print("Asistente: ¡Hasta pronto!")
            break

        if entrada.lower().startswith("ver imagen:"):
            ruta = entrada[11:].strip()
            if not ruta:
                print("Asistente: Por favor, proporciona una ruta válida después de 'ver imagen:'")
                continue

            if not os.path.exists(ruta):
                print("Asistente: No puedo encontrar ese archivo. ¿Estás seguro de que la ruta es correcta?")
                continue

            resultado = asistente.procesar_imagen(ruta_imagen=ruta)
            if resultado.get('estado') == 'éxito':
                print(f"Asistente: He analizado la imagen: {os.path.basename(ruta)}")
            else:
                print(f"Asistente: Hubo un problema al analizar la imagen: {resultado.get('mensaje', 'Error desconocido')}")
        else:
            respuesta = asistente.responder(entrada)
            print(f"Asistente: {respuesta}")

def interfaz_grafica():
    """
    Interfaz gráfica simple para el asistente virtual usando Tkinter
    """
    import tkinter as tk
    from tkinter import filedialog, scrolledtext

    class Aplicacion(tk.Tk):
        def __init__(self):
            super().__init__()
            self.title("Asistente Virtual")
            self.geometry("600x500")
            self.asistente = AsistenteVirtual()

            self.crear_widgets()

        def crear_widgets(self):
            # Área de conversación
            self.conversacion = scrolledtext.ScrolledText(self, wrap=tk.WORD, state='disabled')
            self.conversacion.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

            # Entrada de usuario
            frame_entrada = tk.Frame(self)
            frame_entrada.pack(padx=10, pady=5, fill=tk.X)

            self.entrada = tk.Entry(frame_entrada)
            self.entrada.pack(side=tk.LEFT, fill=tk.X, expand=True)
            self.entrada.bind("<Return>", self.enviar_mensaje)

            tk.Button(frame_entrada, text="Enviar", command=self.enviar_mensaje).pack(side=tk.LEFT, padx=5)
            tk.Button(frame_entrada, text="Cargar Imagen", command=self.cargar_imagen).pack(side=tk.LEFT, padx=5)

            # Mensaje inicial
            self.mostrar_mensaje("Asistente", "¡Hola! Soy tu asistente virtual. Puedo analizar imágenes y conversar contigo.")

        def cargar_imagen(self):
            ruta = filedialog.askopenfilename(
                title="Seleccionar imagen",
                filetypes=[("Imágenes", "*.jpg *.jpeg *.png *.bmp"), ("Todos los archivos", "*.*")]
            )

            if ruta:
                resultado = self.asistente.procesar_imagen(ruta_imagen=ruta)
                if resultado.get('estado') == 'éxito':
                    self.mostrar_mensaje("Asistente", f"He analizado la imagen: {os.path.basename(ruta)}")
                else:
                    self.mostrar_mensaje("Asistente", f"Error al analizar la imagen: {resultado.get('mensaje', 'Error desconocido')}")

        def enviar_mensaje(self, event=None):
            mensaje = self.entrada.get().strip()
            if not mensaje:
                return

            self.mostrar_mensaje("Tú", mensaje)
            self.entrada.delete(0, tk.END)

            respuesta = self.asistente.responder(mensaje)
            self.mostrar_mensaje("Asistente", respuesta)

        def mostrar_mensaje(self, remitente, mensaje):
            self.conversacion.configure(state='normal')
            self.conversacion.insert(tk.END, f"{remitente}: {mensaje}\n\n")
            self.conversacion.configure(state='disabled')
            self.conversacion.see(tk.END)

    app = Aplicacion()
    app.mainloop()

if __name__ == "__main__":
    print("Selecciona el modo de interfaz:")
    print("1. Línea de comandos")
    print("2. Interfaz gráfica")
    opcion = input("Opción (1/2): ").strip()

    if opcion == "1":
        interfaz_cli()
    elif opcion == "2":
        interfaz_grafica()
    else:
        print("Opción no válida. Ejecutando interfaz de línea de comandos por defecto.")
        interfaz_cli()

**Punto 5: Implementación de funcionalidades avanzadas y casos de uso específicos
Finalmente, se implementan las funciones avanzadas y se adapta el asistente para casos de uso específicos.**

In [ ]:
# Implementación de funcionalidades específicas para educación
def generador_ejercicios(tema, dificultad):
    """
    Genera ejercicios educativos basados en un tema y nivel de dificultad
    """
    ejercicios = {
        "matematicas": {
            "facil": ["2 + 2 = ?", "5 × 3 = ?", "10 - 4 = ?"],
            "medio": ["3x + 5 = 20, x = ?", "Área de un cuadrado con lado 5", "√36 = ?"],
            "dificil": ["Derivada de x² + 3x", "Integral de 2x dx", "Límite de (x²-4)/(x-2) cuando x→2"]
        },
        "vocabulario": {
            "facil": ["Sinónimo de 'feliz'", "Antónimo de 'alto'", "Definir 'casa'"],
            "medio": ["Usar 'efímero' en una oración", "Etimología de 'palabra'", "3 palabras que riman con 'sol'"],
            "dificil": ["Diferencia entre 'haber' y 'a ver'", "5 palabras cultas para 'comida'", "Definir 'paronomasia'"]
        }
    }

    if tema in ejercicios and dificultad in ejercicios[tema]:
        return ejercicios[tema][dificultad]
    else:
        return ["Lo siento, no tengo ejercicios para ese tema/dificultad."]

# Implementación de funciones para apoyo psicológico básico
def analisis_sentimiento(texto):
    """
    Realiza un análisis simple de sentimiento en el texto del usuario
    """
    palabras_positivas = {"feliz", "contento", "genial", "maravilloso", "bien", "alegre"}
    palabras_negativas = {"triste", "mal", "deprimido", "ansioso", "preocupado", "enfadado"}

    texto = texto.lower()
    palabras = texto.split()

    positivas = sum(1 for palabra in palabras if palabra in palabras_positivas)
    negativas = sum(1 for palabra in palabras if palabra in palabras_negativas)

    if positivas > negativas:
        return "positivo"
    elif negativas > positivas:
        return "negativo"
    else:
        return "neutral"

# Integración de las funcionalidades avanzadas con el asistente
def mejorar_asistente():
    """
    Mejora el asistente con las nuevas funcionalidades
    """
    # Clase base AsistenteVirtual simulada para el ejemplo
    class AsistenteVirtual:
        def __init__(self):
            self.nombre = "Asistente Virtual Básico"

    # Clase mejorada
    class AsistenteVirtualMejorado(AsistenteVirtual):
        def __init__(self):
            super().__init__()
            self.nombre = "Asistente Virtual Mejorado"
            self.recursos_educativos = {
                "matematicas": ["Khan Academy", "Wolfram Alpha"],
                "ciencias": ["NASA Education", "National Geographic Kids"]
            }

        def analizar_sentimiento_usuario(self, texto):
            return analisis_sentimiento(texto)

        def recomendar_recursos(self, tema, sentimiento=None):
            recursos = self.recursos_educativos.get(tema, ["No tengo recursos para ese tema."])
            if sentimiento == "negativo":
                recursos.append("También recomiendo tomar descansos regulares.")
            return recursos

        def crear_plan_estudio(self, tema, nivel):
            return {
                "tema": tema,
                "nivel": nivel,
                "plan": [
                    f"1. Revisar conceptos básicos de {tema}",
                    f"2. Hacer ejercicios de nivel {nivel}",
                    "3. Revisar errores",
                    "4. Avanzar a temas más complejos"
                ]
            }

    return AsistenteVirtualMejorado

# Casos de uso y ejemplos
def mostrar_casos_uso():
    """
    Muestra ejemplos de cómo utilizar el asistente en diferentes escenarios
    """
    # Caso 1: Apoyo académico
    print("=== Caso de uso: Apoyo académico ===")
    ejercicios = generador_ejercicios("matematicas", "medio")
    print("Ejercicios generados:", ejercicios[:2])

    AsistenteMejorado = mejorar_asistente()
    asistente = AsistenteMejorado()
    plan = asistente.crear_plan_estudio("matematicas", "intermedio")
    print("Plan de estudio:", plan["plan"][0])

    # Caso 2: Apoyo emocional
    print("\n=== Caso de uso: Apoyo emocional ===")
    sentimiento = analisis_sentimiento("Hoy me siento muy feliz y contento")
    print("Análisis de sentimiento:", sentimiento)

    # Caso 3: Recomendaciones personalizadas
    print("\n=== Caso de uso: Recomendaciones personalizadas ===")
    recursos = asistente.recomendar_recursos("ciencias", "positivo")
    print("Recursos recomendados:", recursos[:2])

# Prueba el sistema completo con un ejemplo real
def prueba_sistema_completo():
    """
    Prueba el sistema completo con un ejemplo de conversación
    """
    print("\n=== Prueba del sistema completo ===")
    AsistenteMejorado = mejorar_asistente()
    asistente = AsistenteMejorado()

    # Simular interacción
    usuario = "Estoy estudiando matemáticas pero me siento un poco abrumado"

    # Analizar sentimiento
    sentimiento = asistente.analizar_sentimiento_usuario(usuario)
    print(f"Detecto que tu estado emocional es: {sentimiento}")

    # Recomendar recursos
    if "matemáticas" in usuario.lower():
        recursos = asistente.recomendar_recursos("matematicas", sentimiento)
        print("Te recomiendo estos recursos:", recursos)

    # Generar plan de estudio
    plan = asistente.crear_plan_estudio("matematicas", "principiante")
    print("\nAquí tienes un plan de estudio para empezar:")
    for paso in plan["plan"]:
        print(f"- {paso}")

# Ejecutar demostración
if __name__ == "__main__":
    mostrar_casos_uso()
    prueba_sistema_completo()